In [1]:
// #r "D:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSEC;
using NSE_SIMPLE;
using NSE_SIMPLE.LowMach;
using ilPSP.LinSolvers.PARDISO;
using BoSSS.Solution.Queries;
Init();

The below script needs to be able to find the current output cell; this is an easy method to get it.


(2,1): error CS0006: Metadata file 'BoSSSpad.dll' could not be found

(10,7): error CS0246: The type or namespace name 'ilPSP' could not be found (are you missing a using directive or an assembly reference?)

(11,7): error CS0246: The type or namespace name 'ilPSP' could not be found (are you missing a using directive or an assembly reference?)

(12,7): error CS0246: The type or namespace name 'BoSSS' could not be found (are you missing a using directive or an assembly reference?)

(13,7): error CS0246: The type or namespace name 'BoSSS' could not be found (are you missing a using directive or an assembly reference?)

(14,7): error CS0246: The type or namespace name 'BoSSS' could not be found (are you missing a using directive or an assembly reference?)

(15,7): error CS0246: The type or namespace name 'BoSSS' could not be found (are you missing a using directive or an assembly reference?)

(16,7): error CS0246: The type or namespace name 'BoSSS' could not be found (are you missing a 

Cell not executed: compilation error

In [2]:
static string name = "CTD_CONV_STUDY_PowerLaw";

In [3]:
BoSSSshell.WorkflowMgm.Init(name);


(1,1): error CS0103: The name 'BoSSSshell' does not exist in the current context



Cell not executed: compilation error

In [ ]:
static var myBatch = GetDefaultQueue();
static var myDb = myBatch.CreateOrOpenCompatibleDatabase(name);

In [ ]:
// myBatch.AllowedDatabasesPaths.Add(new AllowedDatabasesPair(myDb.Path,""));

In [ ]:
int[] dgDegree = new int[]{1,2,3,4};
int[] res = new int[]{4,5,6,7};

In [ ]:
Type solverXNSEC = typeof(BoSSS.Application.XNSEC.XNSEC);

for (int dg = 0; dg < dgDegree.Length; dg++) {
    for (int r = 0; r < res.Length; r++) {
        string ending        = "P" + dgDegree[dg] + "K" + res[r];
        string jobName       = string.Format("XNSEC_HEATEDCOUETTE"+ ending);
        string StartupString = string.Format("cs:BoSSS.Application.XNSEC.FullNSEControlExamples.SteadyCouetteFlowWithTemperatureGradient({0}, {1},@\"{2}\")",  dgDegree[dg] ,  res[r], myDb.Path);
        var oneJob           = new Job(jobName, solverXNSEC);
        oneJob.SetControlStatement(StartupString);
        oneJob.NumberOfMPIProcs =  r > 2? 8:4;
        oneJob.Activate(myBatch);
    }
}

In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(PollingIntervallSeconds:5);

In [ ]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable;

In [ ]:
Tab = Tab.ExtractColumns("DGdegree:Velocity*","DGdegree:Temperature", "DGdegree:Pressure","Grid:hMin","SolL2err_u","SolL2err_v","SolL2err_T","SolL2err_p");
var ErrorPlot_u = Tab. ToPlot ("Grid:hMin", "SolL2err_u","DGdegree:Velocity*"); // column for grou
var ErrorPlot_v = Tab. ToPlot ("Grid:hMin", "SolL2err_v","DGdegree:Velocity*"); // column for grou
var ErrorPlot_T = Tab. ToPlot ("Grid:hMin", "SolL2err_T","DGdegree:Temperature"); // column for grou
var ErrorPlot_p = Tab. ToPlot ("Grid:hMin", "SolL2err_p","DGdegree:Pressure"); // column for grou
ErrorPlot_u.LogX = true ;
ErrorPlot_u.LogY = true ;
ErrorPlot_v.LogX = true ;
ErrorPlot_v.LogY = true ;
ErrorPlot_T.LogX = true ;
ErrorPlot_T.LogY = true ;
ErrorPlot_p.LogX = true ;
ErrorPlot_p.LogY = true ;

ConvergencePlots[0,0] = ErrorPlot_u;
ConvergencePlots[0,1] = ErrorPlot_v;
ConvergencePlots[1,0] = ErrorPlot_T;
ConvergencePlots[1,1] = ErrorPlot_p; //   pDegPlot.SaveTextFileToPublish(folderpath+"\\"+  varname + "DG" + pDeg, false);

var gp = ConvergencePlots.ToGnuplot();
gp.PlotSVG(xRes:1000,yRes:800)



In [ ]:
bool savetoTxtFile = false;

In [ ]:
// private static void ToTxtFile(string fileName, double[] x, double[] y) {
//     if(x.Length != y.Length)
//         throw new Exception();
//         using (var file = new StreamWriter(fileName)) {
//             file.Write("x" + "\t"+ "y");
//             file.WriteLine();
//         for(int i = 0; i< x.Length; i++){
//             file.Write(x[i] + "\t" + y[i]);
//             file.WriteLine();
//         }
//     }    
// }

In [ ]:
private static void ToTxtFile(string fileName, double[] x, double[] y) {
    if(x.Length != y.Length)
        throw new Exception();

        var dummy = x.ToList();
        dummy.Reverse();
        var xinv2 = dummy.ToArray();

        var dummy2 = y.ToList();
        dummy2.Reverse();
        var yinv2 = dummy2.ToArray();


        using (var file = new StreamWriter(fileName)) {
            file.Write("x" + "\t"+ "y");
            file.WriteLine();
        for(int i = 0; i< xinv2.Length; i++){
            file.Write(xinv2[i] + "\t" + yinv2[i]);
            file.WriteLine();
        }
    }    
}

In [ ]:
Tab = Tab.ExtractColumns("DGdegree:Velocity*","DGdegree:Temperature", "DGdegree:Pressure","Grid:hMin","SolL2err_u","SolL2err_v","SolL2err_T","SolL2err_p");
string[] varnames = new string[]{"VelocityX","VelocityY","Temperature", "Pressure" };
Plot2Ddata[,] ConvergencePlots = new Plot2Ddata[2, 2];

foreach (var varname in varnames) {
    string SolL2Err = "";
    string DgDegreeStr = "";
    int x = 0;
    int y = 0;
    switch(varname){
        case "VelocityX":
        SolL2Err = "SolL2err_u";
        DgDegreeStr = "DGdegree:Velocity*";
        x = 0; y = 0;
        break;
        case "VelocityY":
        SolL2Err = "SolL2err_v";
        DgDegreeStr = "DGdegree:Velocity*";
        x = 1; y = 0;
        break;
        case "Temperature":
        SolL2Err = "SolL2err_T";
        DgDegreeStr = "DGdegree:Temperature";
        x = 0; y = 1;
        break;
        case "Pressure":
        SolL2Err = "SolL2err_p";
        DgDegreeStr = "DGdegree:Pressure";
        x = 1; y = 1;
        break;
    }

    var ErrorPlot = Tab. ToPlot ("Grid:hMin",SolL2Err,DgDegreeStr); 
    ErrorPlot.LogX = true ;
    ErrorPlot.LogY = true ;
    ConvergencePlots[x,y] = ErrorPlot;
    
    // save text file for each polynomial degree.
    foreach(var datagroup in ErrorPlot.dataGroups ){ 
        int dg =  datagroup.Name[datagroup.Name.Length-1] - '0'; 
            if(varname == "Pressure"){
            dg =dg+1;
        }
        string filename = folderpath + @"\" + varname + "DG" + dg + "Data.txt";
        if(savetoTxtFile)
            ToTxtFile(filename,datagroup.Abscissas,datagroup.Values);
    }
    
}

var gp = ConvergencePlots.ToGnuplot();
gp.PlotSVG(xRes:1000,yRes:800)

In [ ]:
var ErrorPlot_T = Tab. ToPlot ("Grid:hMin", "SolL2err_T","DGdegree:Temperature"); // column for grou
ErrorPlot_T.dataGroups[0].Abscissas

In [ ]:
folderpath

In [ ]:
// string dataPathResultsBase = @"C:\Users\Gutierrez\Documents\data\convergenceProblemData\" ;
string dataPathResultsBase = @"D:\ThesisDGCombustion\dgcombustionthesis\data\ConvStudy_Couette_PowerLaw";
string folderpath          = dataPathResultsBase ;// + ending +"LowTemp"+ @"\";
System.IO.Directory.CreateDirectory(folderpath); // Create folder for storing data